In [1]:
import numpy as np
from multipatch_analysis.database import SynphysDatabase
from multipatch_analysis.cell_class import CellClass, classify_pairs

In [2]:
db = SynphysDatabase(ro_host='sqlite:///', rw_host=None, db_name='/home/luke/public/synphys_current_small.sqlite')

Define two cell classes:
- presynaptic cells are L2/3 pyramidal cells
- postsynaptic cells are Pvalb positive (any layer)

In [3]:
pre_class = CellClass(target_layer='2/3', cre_type='unknown')
post_class = CellClass(cre_type='pvalb')
classes = {
    'L2/3 Sst': pre_class,
    'Pvalb': post_class,
}

Get a list of all cell pairs in the database that match these cell type criteria. We also filter here for a specific project name "mouse V1 coarse matrix" in order to exclude Other experiment types.

In [4]:
pair_query = db.pair_query(project_name="mouse V1 coarse matrix", pre_class=pre_class, post_class=post_class)
pairs = pair_query.all()
len(pairs)

436

The results of this query can also be returned as a pandas dataframe:

In [5]:
pair_df = db.dataframe(pair_query)
pair_df.head()

,id,experiment_id,pre_cell_id,post_cell_id,has_synapse,has_electrical,crosstalk_artifact,n_ex_test_spikes,n_in_test_spikes,distance,meta
0,42172,1202,7199,7201,False,False,None,0,0,0.000090,None
1,42173,1202,7199,7202,False,False,None,0,0,0.000215,None
2,42181,1204,7204,7208,False,False,None,0,0,0.000152,None
3,42184,1204,7204,7209,False,False,None,0,0,0.000143,None
4,42188,1204,7204,7211,False,False,None,0,0,0.000163,None


Of the 374 pairs that were probed in the dataset above, only a fraction of them were found to be connected by a synapse:

In [10]:
connected_mask = pair_df['has_synapse'] == True
synapse_df = pair_df[connected_mask]
synapse_df.head()

,id,experiment_id,pre_cell_id,post_cell_id,has_synapse,has_electrical,crosstalk_artifact,n_ex_test_spikes,n_in_test_spikes,distance,meta
62,59620,1758,10399,10394,True,False,None,1034,480,0.000102,None
65,59635,1758,10401,10394,True,False,None,1041,480,0.000145,None
84,61432,1823,10750,10743,True,False,None,168,36,0.000038,None
97,61797,1837,10820,10821,True,False,None,946,72,0.000053,None
112,61864,1839,10833,10838,True,False,None,1441,731,0.000131,None


In [15]:
amps = [pair.synapse.psp_amplitude for pair in pairs if pair.has_synapse]
amps = [amp for amp in amps if amp is not None]
print("Average PSP amplitude for %s->%s: %0.2f mV" % (pre_class, post_class, np.mean(amps)*1e3))

Average PSP amplitude for L2/3 unknown->pvalb: 0.33 mV
